# 필요 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

import random
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action = 'ignore')

# 데이터 불러오기

In [ ]:
data = pd.read_csv('../data/recomm/user_item_interactions.csv')
data = data.rename(columns={'TRAVELER_ID': 'user_id', 'POI_ID': 'item_id'})
display(data.head())

display(data.info())

In [ ]:
data_user_ids = data['user_id'].unique()
data_item_ids = data['item_id'].unique()
print("훈련 데이터셋에 있는 유저 수:", len(data_user_ids), "훈련 데이터셋에 있는 아이템 수:", len(data_item_ids))

In [ ]:
# POIMeta2vec 임베딩 벡터 로드
poimeta2vec_df = pd.read_csv('../model/poimeta2vec_df.csv', index_col=0, dtype=str)

# 데이터 전처리

## Train & Test split

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data.values, test_size = 0.2, random_state = 42)
train = pd.DataFrame(train, columns = data.columns)
test = pd.DataFrame(test, columns = data.columns)

In [ ]:
print('Train size : ', len(train))
print('Test size : ', len(test))

## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

user_label_encoder = LabelEncoder()
item_label_encoder = LabelEncoder()

train['user_id_idx'] = user_label_encoder.fit_transform(train['user_id'].values)
train['item_id_idx'] = item_label_encoder.fit_transform(train['item_id'].values)

In [ ]:
train_user_ids = train['user_id'].unique()
train_item_ids = train['item_id'].unique()
print("훈련 데이터셋에 있는 유저 수:", len(train_user_ids), "훈련 데이터셋에 있는 아이템 수:", len(train_item_ids))

In [ ]:
test_user_ids = test['user_id'].unique()
test_item_ids = test['item_id'].unique()
print("테스트 데이터셋에 있는 유저 수:", len(test_user_ids), "테스트 데이터셋에 있는 아이템 수:", len(test_item_ids))

In [ ]:
test = test[(test['user_id'].isin(train_user_ids)) & (test['item_id'].isin(train_item_ids))]
test['user_id_idx'] = user_label_encoder.transform(test['user_id'].values)
test['item_id_idx'] = item_label_encoder.transform(test['item_id'].values)
print("테스트 데이터셋 크기:", len(test))

# 모델 구축

In [ ]:
# dok_mtrx를 COO 형식으로 변환하여 희소 텐서로 변환하는 함수
def convert_to_sparse_tensor(dok_mtrx):
    # dok_mtrx를 COO 형식으로 변환
    dok_mtrx_coo = dok_mtrx.tocoo().astype(np.float32)

    # COO 형식으로 변환된 행렬에서 값과 인덱스 추출
    values = dok_mtrx_coo.data
    indices = np.vstack((dok_mtrx_coo.row, dok_mtrx_coo.col))

    # 인덱스를 torch.LongTensor로 변환하고 값은 torch.FloatTensor로 변환
    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)

    # 행렬의 모양 추출
    shape = dok_mtrx_coo.shape

    # 변환된 값과 인덱스를 사용하여 희소 텐서 생성
    dok_mtrx_sparse_tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))

    # 생성된 희소 텐서 반환
    return dok_mtrx_sparse_tensor

In [ ]:
# cpu연산용

import scipy.sparse as sparse

def get_metrics(W_u, W_i, n_users, n_items, train_data, test_data, K):
    test_user_ids = torch.LongTensor(test_data['user_id_idx'].unique())
    relevance_score = torch.matmul(W_u, torch.transpose(W_i, 0, 1))
    R = sparse.dok_matrix((n_users, n_items), dtype=np.float32)
    R[train_data['user_id_idx'], train_data['item_id_idx']] = 1.0
    R_tensor = convert_to_sparse_tensor(R)
    R_tensor_dense = R_tensor.to_dense()
    R_tensor_dense = R_tensor_dense * (-np.inf)
    R_tensor_dense = torch.where(torch.isnan(R_tensor_dense), torch.full_like(R_tensor_dense, 0), R_tensor_dense)
    relevance_score = relevance_score + R_tensor_dense
    topk_idx = torch.topk(relevance_score, K).indices
    topk_idx_df = pd.DataFrame(topk_idx.numpy(), columns=['top_idx' + str(x + 1) for x in range(K)])
    topk_idx_df['user_id'] = topk_idx_df.index
    topk_idx_df['topk_item'] = topk_idx_df[['top_idx' + str(x + 1) for x in range(K)]].values.tolist()
    topk_idx_df = topk_idx_df[['user_id', 'topk_item']]
    test_items = test_data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()
    metrics_df = pd.merge(test_items, topk_idx_df, how='left', left_on='user_id_idx', right_on='user_id')
    metrics_df['interact_items'] = [list(set(a).intersection(b)) for a, b in
                                    zip(metrics_df.item_id_idx, metrics_df.topk_item)]
    metrics_df['recall'] = metrics_df.apply(lambda x: len(x['interact_items']) / len(x['item_id_idx']), axis=1)

    def get_dcg_idcg(item_id_idx, hit_list):
        dcg = sum([hit * np.reciprocal(np.log1p(idx + 1)) for idx, hit in enumerate(hit_list)])
        idcg = sum([np.reciprocal(np.log1p(idx + 1)) for idx in range(min(len(item_id_idx), len(hit_list)))])
        return dcg / idcg

    metrics_df['hit_list'] = metrics_df.apply(lambda x: [1 if i in set(x['item_id_idx']) else 0 for i in x['topk_item']],
                                              axis=1)
    metrics_df['ndcg'] = metrics_df.apply(lambda x: get_dcg_idcg(x['item_id_idx'], x['hit_list']), axis=1)
    return metrics_df['recall'].mean(), metrics_df['ndcg'].mean()

In [ ]:
class LightGCN(nn.Module):
    def __init__(self, data, n_users, n_items, n_layers, latent_dim, poimeta2vec_df):
        super(LightGCN, self).__init__()
        self.data = data
        self.n_users = n_users
        self.n_items = n_items
        self.n_layers = n_layers
        self.latent_dim = latent_dim
        self.poimeta2vec_df = poimeta2vec_df
        self.init_embedding()
        self.norm_adj_mat_sparse_tensor = self.get_norm_adj()

    def init_embedding(self):
        self.E0 = nn.Embedding(self.n_users + self.n_items, self.latent_dim)
        nn.init.xavier_uniform_(self.E0.weight)
        self.E0.weight = nn.Parameter(self.E0.weight)

    def get_norm_adj(self):
        R = sparse.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        R[self.data['user_id_idx'], self.data['item_id_idx']] = 1.0
        adj_mat = sparse.dok_matrix(
            (self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32)
        adj_mat[:self.n_users, self.n_users:] = R
        adj_mat[self.n_users:, :self.n_users] = R.T
        rowsum = np.array(adj_mat.sum(1))
        d_inv = np.power(rowsum + 1e-9, -0.5).flatten()
        d_mat_inv = sparse.diags(d_inv)
        norm_adj_mat = d_mat_inv.dot(adj_mat).dot(d_mat_inv)
        norm_adj_mat_coo = norm_adj_mat.tocoo().astype(np.float32)
        values = norm_adj_mat_coo.data
        indices = np.vstack((norm_adj_mat_coo.row, norm_adj_mat_coo.col))
        i = torch.LongTensor(indices)
        v = torch.FloatTensor(values)
        return torch.sparse.FloatTensor(i, v, torch.Size(norm_adj_mat_coo.shape))

    def propagate_through_layers(self):
        all_layers_embedding = [self.E0.weight]
        E_lyr = self.E0.weight
        for layer in range(self.n_layers):
            E_lyr = torch.sparse.mm(self.norm_adj_mat_sparse_tensor, E_lyr)
            all_layers_embedding.append(E_lyr)
        all_layers_embedding = torch.stack(all_layers_embedding)
        mean_layer_embedding = torch.mean(all_layers_embedding, axis=0)
        return torch.split(mean_layer_embedding, [self.n_users, self.n_items]) + torch.split(self.E0.weight, [self.n_users, self.n_items])

    def forward(self, users, pos_items, neg_items, poimeta2vec_tensor):
        final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = self.propagate_through_layers()
        pos_poimeta2vec_emb = poimeta2vec_tensor[pos_items]
        neg_poimeta2vec_emb = poimeta2vec_tensor[neg_items]
        final_item_emb_pos = final_item_emb[pos_items] + pos_poimeta2vec_emb
        final_item_emb_neg = final_item_emb[neg_items] + neg_poimeta2vec_emb
        initial_item_emb_pos = initial_item_emb[pos_items] + pos_poimeta2vec_emb
        initial_item_emb_neg = initial_item_emb[neg_items] + neg_poimeta2vec_emb
        return (final_user_emb[users], final_item_emb_pos, final_item_emb_neg, initial_user_emb[users], initial_item_emb_pos, initial_item_emb_neg)

    def get_poimeta2vec_emb(self, item_ids):
        poimeta2vec_emb = []
        for item_id in item_ids:
            item_poimeta2vec = self.poimeta2vec_df.loc[self.poimeta2vec_df['POI_ID'] == item_id]
            if len(item_poimeta2vec) > 0:
                poimeta2vec_emb.append(item_poimeta2vec.values[0][1:])
            else:
                # 해당 item_id에 대한 POIMeta2vec 임베딩이 없는 경우 랜덤 값으로 초기화
                poimeta2vec_emb.append(np.random.normal(size=self.latent_dim))
        return torch.tensor(poimeta2vec_emb, dtype=torch.float32)

In [ ]:
# cpu 연산용
def bpr_loss(users, users_emb, pos_emb, neg_emb, user_emb_0, pos_emb_0, neg_emb_0, config):
    pos_scores = torch.sum(torch.mul(users_emb, pos_emb), dim=1)
    neg_scores = torch.sum(torch.mul(users_emb, neg_emb), dim=1)
    loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores)))
    if config.reg > 0:
        l2_norm = (user_emb_0.norm().pow(2) + pos_emb_0.norm().pow(2) + neg_emb_0.norm().pow(2)) / float(len(users))
        reg_loss = config.reg * l2_norm
        loss += reg_loss
    return loss

# 모델학습

In [ ]:
n_users = train['user_id_idx'].nunique()
n_items = train['item_id_idx'].nunique()

In [ ]:
from sklearn.decomposition import PCA

# cpu연산용
def data_loader(data, batch_size, n_users, n_items, poimeta2vec_df, latent_dim):
    interacted_items_df = data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()

    def sample_neg(x):
        while True:
            neg_id = random.randint(0, n_items - 1)
            if neg_id not in x:
                return neg_id

    indices = [x for x in range(n_users)]
    if n_users < batch_size:
        users = [random.choice(indices) for _ in range(batch_size)]
    else:
        users = random.sample(indices, batch_size)
    users.sort()
    users_df = pd.DataFrame(users, columns=['users'])
    interacted_items_df = pd.merge(interacted_items_df, users_df, how='right', left_on='user_id_idx',
                                   right_on='users')
    
    pos_items = interacted_items_df['item_id_idx'].apply(lambda x: random.choice(x) if isinstance(x, list) else -1).values
    neg_items = interacted_items_df['item_id_idx'].apply(lambda x: sample_neg(x) if isinstance(x, list) else -1).values
    
    # NaN 값을 -1로 대체
    pos_items = np.where(np.isnan(pos_items), -1, pos_items).astype(int)
    neg_items = np.where(np.isnan(neg_items), -1, neg_items).astype(int)
    
    poimeta2vec_emb_dim = poimeta2vec_df.shape[1] - 1
    if poimeta2vec_emb_dim != latent_dim:
        poimeta2vec_emb = poimeta2vec_df.drop('POI_ID', axis=1).values
        if poimeta2vec_emb_dim > latent_dim:
            pca = PCA(n_components=latent_dim)
            poimeta2vec_emb = pca.fit_transform(poimeta2vec_emb)
        else:
            poimeta2vec_emb = np.hstack((poimeta2vec_emb, np.random.normal(size=(poimeta2vec_emb.shape[0], latent_dim - poimeta2vec_emb_dim))))
        poimeta2vec_df = pd.concat([poimeta2vec_df['POI_ID'], pd.DataFrame(poimeta2vec_emb)], axis=1)
    poimeta2vec_tensor = torch.tensor(poimeta2vec_df.drop('POI_ID', axis=1).values, dtype=torch.float32)
    return list(users), list(pos_items), list(neg_items), poimeta2vec_tensor

## 하이퍼 파라미터 튜닝

In [ ]:
import optuna
import math
from box import Box

# 목적 함수: Optuna 실험을 위한
def objective(trial, poimeta2vec_df, train, test, n_users, n_items):
    # 하이퍼파라미터 설정
    config = {
        'latent_dim': trial.suggest_categorical('latent_dim', [32, 64, 128]),  # 임베딩 차원
        'lr': trial.suggest_loguniform('lr', 1e-4, 1e-2),  # 학습률
        'batch_size': trial.suggest_categorical('batch_size', [64, 128, 256]),  # 배치 크기
        'num_layers': trial.suggest_int('num_layers', 1, 3),  # 그래프 컨볼루션 층의 수
        'reg': trial.suggest_loguniform('reg', 1e-6, 1e-3),  # 정규화 강도
        'epochs': 20,  # 에폭 수
        'top_k': 10,  # 상위 K개의 추천
        'patience': 5  # 조기 종료 기준
    }
    config = Box(config)  # 설정을 쉽게 접근하기 위해 Box 객체 사용
    # 학습 데이터 총 개수를 배치 크기로 나누어 총 배치 수 계산
    n_batch = math.ceil(len(train) / config['batch_size'])

    # LightGCN 모델 초기화
    model = LightGCN(train, n_users, n_items, config['num_layers'], config['latent_dim'], poimeta2vec_df)

    # 옵티마이저 설정: Adam 알고리즘 사용
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['reg'])

    # 조기 종료를 위한 카운터와 최고 테스트 손실값 초기화
    early_stop_counter = 0
    best_test_loss = float('inf')

    # 설정된 에폭 수만큼 반복
    for epoch in range(config.epochs):
        model.train()  # 모델을 학습 모드로 설정
        train_loss_epoch = []  # 에폭별 학습 손실을 저장할 리스트

        # 각 배치에 대해 반복
        for batch_idx in range(n_batch):
            optimizer.zero_grad()  # 그라디언트 초기화

            # 데이터 로더를 통해 현재 배치의 사용자, 긍정 아이템, 부정 아이템, 메타데이터 임베딩 텐서 가져오기
            users, pos_items, neg_items, poimeta2vec_tensor = data_loader(train, config.batch_size, n_users, n_items, poimeta2vec_df, config.latent_dim)

            # 모델의 포워드 패스 실행
            users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0 = model(users, pos_items, neg_items, poimeta2vec_tensor)

            # BPR 손실 계산
            train_loss = bpr_loss(users, users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0, config)
            train_loss.backward()  # 손실에 대해 역전파 수행
            optimizer.step()  # 옵티마이저를 통해 파라미터 업데이트
            train_loss_epoch.append(train_loss.item())  # 현재 배치의 손실을 저장

        model.eval()  # 모델을 평가 모드로 설정
        with torch.no_grad():  # 그라디언트 계산을 비활성화하여 메모리 사용 최소화
            # 모델을 통해 사용자와 아이템의 최종 임베딩을 얻음
            final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = model.propagate_through_layers()

            # 테스트 데이터에 대한 메트릭 계산
            test_recall, test_ndcg = get_metrics(final_user_emb, final_item_emb, n_users, n_items, train, test, config['top_k'])

            # 테스트 데이터 로딩
            test_users, test_pos_items, test_neg_items, test_poimeta2vec_tensor = data_loader(test, config.batch_size, n_users, n_items, poimeta2vec_df, config.latent_dim)

            # 테스트 데이터에 대해 모델 포워드 패스 실행
            test_users_emb, test_pos_emb, test_neg_emb, test_users_emb_0, test_pos_emb_0, test_neg_emb_0 = model(test_users, test_pos_items, test_neg_items, test_poimeta2vec_tensor)

            # 테스트 손실 계산
            test_loss = bpr_loss(test_users, test_users_emb, test_pos_emb, test_neg_emb, test_users_emb_0, test_pos_emb_0, test_neg_emb_0, config)

        # 에폭별 결과 출력
        print(f"Epoch {epoch+1}/{config['epochs']}, Train Loss: {np.mean(train_loss_epoch):.4f}, Test Loss: {test_loss:.4f}, Recall@{config['top_k']}: {test_recall:.4f}, NDCG@{config['top_k']}: {test_ndcg:.4f}")

        # 조기 종료 로직
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            early_stop_counter = 0  # 최고 손실이 개선되면 카운터를 리셋
        else:
            early_stop_counter += 1
            if early_stop_counter >= config.patience:  # 조기 종료 기준 도달 시
                print(f"Early stopping at epoch {epoch+1}")
                break  # 학습 중단

    return best_test_loss

# 하이퍼파라미터 튜닝 수행
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, poimeta2vec_df, train, test, n_users, n_items), n_trials=30)

print("Best hyperparameters:", study.best_params)
print("Best test loss:", study.best_value)

In [ ]:
"""
optuna 로 최적 모델을 탐색했으나 커널이슈로 기록이 사라짐
이슈 발생전 기록해놓은 최적 파라미터값은 아래와 같음
"""
# 최적 파라미터로 50 epch를 학습한 모델 저장
# Trial 15 finished with value: 0.2492541342615162 and parameters: {'latent_dim': 64, 'lr': 0.009631732140395756, 
#'batch_size': 256, 'num_layers': 4, 'reg': 1.0794690314138785e-06}. Best is trial 15 with value: 0.2492541342615162.

## 베스트 하이퍼파라미터 학습(w. 앙상블)

In [ ]:
# 앙상블용
def bpr_loss(users, users_emb, pos_emb, neg_emb, user_emb_0, pos_emb_0, neg_emb_0, params):
    pos_scores = torch.sum(torch.mul(users_emb, pos_emb), dim=1)
    neg_scores = torch.sum(torch.mul(users_emb, neg_emb), dim=1)
    loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores)))
    if params['reg'] > 0:
        l2_norm = (user_emb_0.norm().pow(2) + pos_emb_0.norm().pow(2) + neg_emb_0.norm().pow(2)) / float(len(users))
        reg_loss = params['reg'] * l2_norm
        loss += reg_loss
    return loss

In [ ]:
import math
from sklearn.model_selection import KFold

# 최적 하이퍼파라미터 설정
best_params_list = [
    {'latent_dim': 128, 'lr': 0.0012643544834896815, 'batch_size': 64, 'num_layers': 3, 'reg': 3.3976282399703575e-06},  # Trial 13
    {'latent_dim': 128, 'lr': 0.003323105085677813, 'batch_size': 64, 'num_layers': 3, 'reg': 2.9697461259314475e-06},   # Trial 18
    {'latent_dim': 128, 'lr': 0.00023329374686282962, 'batch_size': 64, 'num_layers': 2, 'reg': 7.551153211170191e-06},  # Trial 6
    {'latent_dim': 128, 'lr': 0.0028407241056413366, 'batch_size': 128, 'num_layers': 3, 'reg': 3.531882181203301e-06}   # Trial 15
]

# K-Fold 교차 검증 설정
n_splits = 5
kf = KFold(n_splits=n_splits)

# 앙상블에 사용될 모델들을 저장할 리스트
models = []

# K-Fold 교차 검증 수행
for fold, (train_idx, val_idx) in enumerate(kf.split(train)):
    print(f"Fold {fold+1}/{n_splits}")
    
    train_fold, val_fold = train.iloc[train_idx], train.iloc[val_idx]
    
    for best_params in best_params_list:
        best_params['epochs'] = 200  # 최종 학습을 위해 에포크 수 조정
        best_params['patience'] = 10  # 조기 종료를 위한 patience 값 설정
        
        # poimeta2vec_df를 DataFrame 형태로 전달
        lightGCN = LightGCN(train_fold, n_users, n_items, best_params['num_layers'], best_params['latent_dim'], poimeta2vec_df)
        optimizer = torch.optim.Adam(lightGCN.parameters(), lr=best_params['lr'], weight_decay=best_params['reg'])
        
        top_k = 10  # top_k 값을 직접 설정
        
        loss_list_epoch = []
        recall_list = []
        ndcg_list = []
        
        n_batch = math.ceil(len(train_fold) / best_params['batch_size'])  # train_dataset의 총 개수에 따라 조정 필요
        
        early_stop_counter = 0
        best_val_loss = float('inf')
        
        for epoch in range(best_params['epochs']):
            lightGCN.train()
            final_loss_list = []
            
            for batch_idx in range(n_batch):
                optimizer.zero_grad()
                users, pos_items, neg_items, poimeta2vec_tensor = data_loader(train_fold, best_params['batch_size'], n_users, n_items, poimeta2vec_df, best_params['latent_dim'])
                users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0 = lightGCN(users, pos_items, neg_items, poimeta2vec_tensor)
                final_loss = bpr_loss(users, users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0, best_params)
                final_loss.backward()
                optimizer.step()
                final_loss_list.append(final_loss.item())
            
            lightGCN.eval()
            with torch.no_grad():
                final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = lightGCN.propagate_through_layers()
                val_users, val_pos_items, val_neg_items, val_poimeta2vec_tensor = data_loader(val_fold, best_params['batch_size'], n_users, n_items, poimeta2vec_df, best_params['latent_dim'])
                val_users_emb, val_pos_emb, val_neg_emb, val_users_emb_0, val_pos_emb_0, val_neg_emb_0 = lightGCN(val_users, val_pos_items, val_neg_items, val_poimeta2vec_tensor)
                val_loss = bpr_loss(val_users, val_users_emb, val_pos_emb, val_neg_emb, val_users_emb_0, val_pos_emb_0, val_neg_emb_0, best_params)
                val_recall, val_ndcg = get_metrics(final_user_emb, final_item_emb, n_users, n_items, train_fold, val_fold, top_k)
            
            loss_list_epoch.append(np.mean(final_loss_list))
            recall_list.append(val_recall)
            ndcg_list.append(val_ndcg)
            
            print(f"[에폭: {epoch+1}/{best_params['epochs']}, 손실: {np.mean(final_loss_list):.4f}, 검증 손실: {val_loss:.4f}, 리콜@{top_k}: {val_recall:.4f}, NDCG@{top_k}: {val_ndcg:.4f}]")
            
            # 조기 종료 로직
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                early_stop_counter = 0
            else:
                early_stop_counter += 1
            
            if early_stop_counter >= best_params['patience']:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        models.append(lightGCN)

# 앙상블 평가
ensemble_recall, ensemble_ndcg = ensemble_evaluate(models, test, top_k)
print(f"앙상블 결과 - 리콜@{top_k}: {ensemble_recall:.4f}, NDCG@{top_k}: {ensemble_ndcg:.4f}")

## 최종모델 학습

- 앙상블 없이 Best Parameter만으로 학습하려는 경우 사용

In [ ]:
# 최적 하이퍼파라미터 설정
#best_params = study.best_params


# 학습 이력을 바탕으로 확인한 최적의 하이퍼파라미터 설정
best_params = {
    'latent_dim': 64,  # 최적의 latent dimension 크기
    'lr': 0.009631732140395756,  # 최적의 learning rate
    'batch_size': 256,  # 최적의 batch size
    'num_layers': 4,  # 최적의 layer 수
    'reg': 1.0794690314138785e-06,  # 최적의 regularization strength
    'epochs': 200  # 최종 학습을 위해 조정한 에포크 수
}


best_params['epochs'] = 200  # 최종 학습을 위해 에포크 수 조정

# 최적 하이퍼파라미터로 모델 학습
lightGCN = LightGCN(train, n_users, n_items, best_params['num_layers'], best_params['latent_dim'], poimeta2vec_df)
optimizer = torch.optim.Adam(lightGCN.parameters(), lr=best_params['lr'], weight_decay=best_params['reg'])

In [ ]:
import math

top_k = 10  # top_k 값을 직접 설정

loss_list_epoch = []
recall_list = []
ndcg_list = []
n_batch = math.ceil(len(train) / best_params['batch_size'])  # train_dataset의 총 개수에 따라 조정 필요

for epoch in range(best_params['epochs']):
    lightGCN.train()
    final_loss_list = []
    for batch_idx in range(n_batch):
        optimizer.zero_grad()
        users, pos_items, neg_items, poimeta2vec_df = data_loader(train, best_params['batch_size'], n_users, n_items, poimeta2vec_df, best_params['latent_dim'])
        users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0 = lightGCN.forward(users, pos_items, neg_items)
        final_loss = bpr_loss(users, users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0)
        final_loss.backward()
        optimizer.step()
        final_loss_list.append(final_loss.item())
    
    lightGCN.eval()
    with torch.no_grad():
        final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = lightGCN.propagate_through_layers()
        test_recall, test_ndcg = get_metrics(final_user_emb, final_item_emb, n_users, n_items, train, test, top_k)  # best_params['top_k'] 대신 직접 설정한 top_k 사용
    
    loss_list_epoch.append(np.mean(final_loss_list))
    recall_list.append(test_recall)
    ndcg_list.append(test_ndcg)
    
    print(f"[에폭: {epoch+1}/{best_params['epochs']}, 손실: {np.mean(final_loss_list):.4f}, 리콜@{top_k}: {test_recall:.4f}, NDCG@{top_k}: {test_ndcg:.4f}]")

In [ ]:
torch.save(lightGCN.state_dict(), '../model/LightGCN_best_model_weights.pth')

# 학습 결과 평가

In [ ]:
import matplotlib.pyplot as plt

# 학습 결과 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(loss_list_epoch)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 3, 2)
plt.plot(recall_list)
plt.title('Test Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')

plt.subplot(1, 3, 3)
plt.plot(ndcg_list)
plt.title('Test NDCG')
plt.xlabel('Epoch')
plt.ylabel('NDCG')

plt.tight_layout()
plt.show()

# 추천여행지 추출

In [ ]:
# 모델 가중치 저장
torch.save(lightGCN.state_dict(), '../model/LightGCN_best_model_weights.pth')

In [ ]:
# 관광 메타 데이터 
df = pd.read_csv('../data/travel_meta/travel_meta.csv', encoding = 'utf-8-sig')

In [ ]:
import pandas as pd

# 사용자 ID 입력받기
user_id = input("사용자 ID를 입력하세요: ")
user_idx = user_label_encoder.transform([user_id])[0]

lightGCN.eval()
with torch.no_grad():
    # 사용자 임베딩 가져오기
    final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = lightGCN.propagate_through_layers()
    user_embedding = final_user_emb[user_idx]

    # 아이템별 유사도 점수 계산
    similarity_scores = torch.matmul(user_embedding, final_item_emb.transpose(0, 1))
    recommended_item_indices = torch.topk(similarity_scores, top_k).indices

# 추천된 아이템의 상세 정보를 포함한 데이터 프레임 생성
recommended_items = []

# 추천된 아이템들의 상세 정보 추출
for item_idx in recommended_item_indices:
    item_id = item_label_encoder.inverse_transform([item_idx.item()])[0]
    item_info = df[df['POI_ID'] == item_id][['POI_ID', 'POI_NM', 'POI_TYPE']].iloc[0]
    # item_info에 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'STARS' 정보 추가
    item_info['REVISIT_INTENTION'] = data[data['item_id'] == item_id]['REVISIT_INTENTION'].iloc[0]
    item_info['RCMDTN_INTENTION'] = data[data['item_id'] == item_id]['RCMDTN_INTENTION'].iloc[0]
    item_info['STARS'] = data[data['item_id'] == item_id]['STARS'].iloc[0]
    recommended_items.append(item_info)

# 리스트를 데이터프레임으로 변환
recommended_items_df = pd.concat(recommended_items, axis=1).transpose()

# 결과 출력
print(f"사용자 {user_id}에 대한 Top-{top_k} 추천 아이템:")
display(recommended_items_df)

# FastAPI 코드

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import torch

app = FastAPI()

class UserIDRequest(BaseModel):
    user_id: str

@app.post("/recommend/")
async def recommend(request: UserIDRequest):
    user_id = request.user_id
    user_idx = user_label_encoder.transform([user_id])[0]

    lightGCN.eval()
    with torch.no_grad():
        # 사용자 임베딩 가져오기
        final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = lightGCN.propagate_through_layers()
        user_embedding = final_user_emb[user_idx]

        # 아이템별 유사도 점수 계산
        similarity_scores = torch.matmul(user_embedding, final_item_emb.transpose(0, 1))
        recommended_item_indices = torch.topk(similarity_scores, 5).indices  # top_k를 5로 가정

    recommended_items = []

    for item_idx in recommended_item_indices:
        item_id = item_label_encoder.inverse_transform([item_idx.item()])[0]
        item_info = df[df['POI_ID'] == item_id][['POI_ID', 'POI_NM', 'POI_TYPE']].iloc[0].to_dict()
        item_info['REVISIT_INTENTION'] = data[data['item_id'] == item_id]['REVISIT_INTENTION'].iloc[0]
        item_info['RCMDTN_INTENTION'] = data[data['item_id'] == item_id]['RCMDTN_INTENTION'].iloc[0]
        item_info['STARS'] = data[data['item_id'] == item_id]['STARS'].iloc[0]
        recommended_items.append(item_info)

    return {"user_id": user_id, "recommendations": recommended_items}
